#### NLTK Text Pre-processing:

In [4]:
import os
import json
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from gensim.models import Word2Vec
from cp_words import compound_keywords

In [5]:
def preprocess_text(text):
    # Define a list of English stopwords
    stop_words = set(stopwords.words('english'))
    # Add specific terms to be removed
    remove_terms = {'introduct', 'literature', 'review', 'figure', 'doi', 'fig', 'table', 'conclusion', 
                    'altimg', 'gif', 'png', 'discussion', 'acknowledgment', 'appendix','http', 'copyright'}
    stop_words.update(remove_terms)

    # Initialize lemmatizer and stemmer
    lemmatizer = WordNetLemmatizer()
    stemmer = PorterStemmer()

    # Convert to lowercase
    text = text.lower()
    for original, compound in compound_keywords.items():
        text = text.replace(original, compound)
    
    # Tokenize the text into sentences
    sentence_tokens = sent_tokenize(text)
    sentences = []

    # Tokenize each sentence into words and remove stopwords
    for sentence in sentence_tokens:
        words = word_tokenize(sentence)
        filtered_words = []
        for word in words:
            word = lemmatizer.lemmatize(word)
            #word = stemmer.stem(word)
            
            # Remove non-alphabetic characters except underscores and hyphens
            word = re.sub(r'[^\w\s\-]', '', word)
            
            #if word.lower() not in stop_words:
            if word.lower() not in stop_words and not word.isdigit() and 1 < len(word) <= 20:
                filtered_words.append(word)
            
        sentences.append(filtered_words)
    
    return sentences

def process_json_files(directory):
    all_sentences = []

    # Iterate through each JSON file in the directory
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            file_path = os.path.join(directory, filename)
            with open(file_path, 'r', encoding='utf-8') as file:
                data = json.load(file)
                
                # Extract the original text from the JSON structure
                text = data['full-text-retrieval-response']['originalText']
                # Process the text to tokenize and remove stopwords
                processed_sentences = preprocess_text(text)
                # Append the processed sentences to the overall list
                all_sentences.extend(processed_sentences)
    
    return all_sentences

#### Train and Save Word2Vec Model:

In [ ]:
directory = 'downloaded_articles'

model = Word2Vec(
    sentences=process_json_files(directory),  
    vector_size=300,           # Size of the embedding vectors
    window=20,                 # Context window size
    min_count=2,               # Minimum occurrence in vocabulary
    workers=4                  # Number of threads for model training
)

In [20]:
model_save_path = "word2vec_model.model"
model.save(model_save_path)
model = Word2Vec.load("word2vec_model.model")
print("Model loaded successfully.")

Model loaded successfully.


#### Model Examination:

In [19]:
# Find similar words 
similar_words_rl = model.wv.most_similar('rl', topn=100)
similar_words_carbon = model.wv.most_similar('carbon', topn=20)

print("Words similar to 'rl':")
for word, similarity in similar_words_rl:
    print(f"{word}: {similarity}")

print("\nWords similar to 'carbon':")
for word, similarity in similar_words_carbon:
    print(f"{word}: {similarity}")

Words similar to 'rl':
drl: 0.7762457728385925
model-free_rl: 0.7014486193656921
model-based_rl: 0.6874682307243347
rl-based: 0.6831478476524353
marl: 0.629423201084137
dqn: 0.6275638937950134
ppo: 0.6267271637916565
sac: 0.6256202459335327
ddpg: 0.6214520335197449
ddq: 0.6198261380195618
single-agent: 0.6155815124511719
mcts: 0.6127952933311462
irl: 0.6084239482879639
policy-based: 0.6080467104911804
maddpg: 0.6001636981964111
learns: 0.597644567489624
value-based: 0.5944128036499023
drl-based: 0.5910117626190186
on-policy: 0.5886929631233215
agent: 0.5880126357078552
learn: 0.5829527974128723
ddqn: 0.5671823024749756
unsl: 0.5661435127258301
behavioral_cloning: 0.564849317073822
dyna-pinn: 0.5625392198562622
mdp: 0.5622559785842896
sarl: 0.5586027503013611
model-free: 0.5563459992408752
policy: 0.5514605641365051
dyna-style: 0.5495252013206482
dnns: 0.5471894145011902
gail: 0.5425839424133301
bdq: 0.5411962270736694
pomdp: 0.5403475165367126
q-learning: 0.5390430092811584
pre-trainin

In [21]:
# Find similarity
similarity_1 = model.wv.similarity('build', 'hous')
print(similarity_1)

0.23131573
